# 設定

In [10]:
import os
from dotenv import load_dotenv
import time
load_dotenv()

BITMART_API_KEY = os.getenv("BITMART_API_KEY")
BITMART_SECRET_KEY = os.getenv("BITMART_SECRET_KEY")
BITMART_MEMO = os.getenv("BITMART_MEMO")
TOPONE_API_KEY = os.getenv("TOPONE_API_KEY")
TOPONE_SECRET_KEY = os.getenv("TOPONE_SECRET_KEY")

# bitmart 客戶端


In [11]:
import logging
import time
import pandas as pd
from bitmart.api_contract import APIContract
from bitmart.lib.cloud_exceptions import APIException
import math
import uuid
import datetime
import requests
from datetime import timedelta,datetime

# 設定 logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger("bitmart").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

class BitmartClient:
    def __init__(self, api_key: str, secret_key: str, memo: str):
        self.futuresAPI = APIContract(api_key=api_key, secret_key=secret_key, memo=memo)
        self.logger = logging.getLogger(__name__)
        self.BASE_URL = "https://api-cloud-v2.bitmart.com"
    
    def _time_to_timestamp(self, t):
        """將 datetime 或 timestamp 轉成秒級 timestamp"""
        if t is None:
            return int(time.time())
        elif isinstance(t, (int, float)):
            return int(t)
        elif isinstance(t, datetime):
            return int(t.timestamp())
        else:
            raise ValueError("❌ start_time/end_time 必須是 datetime 或 timestamp 或 None")

    def get_balance(self):
        try:
            response = self.futuresAPI.get_assets_detail()[0]
            usdt_asset = next((asset for asset in response['data'] if asset['currency'] == 'USDT'), None)
            if usdt_asset:
                return float(usdt_asset['available_balance'])
            else:
                self.logger.info("USDT asset not found")
                return None
        except APIException as error:
            self.logger.error(f"Found error. status: {error.status_code}, error message: {error.response}")
            return None
    
    def get_current_price(self, symbol: str):
        try:
            depth_data = self.futuresAPI.get_depth(symbol)[0]['data']
            if depth_data and depth_data.get('bids') and depth_data.get('asks'):
                best_bid = float(depth_data['bids'][0][0])
                best_ask = float(depth_data['asks'][0][0])
                return (best_bid + best_ask) / 2
            else:
                self.logger.error(f"Could not get bids/asks from depth data.")
                return None
        except (APIException, IndexError, KeyError) as error:
            self.logger.error(f"Failed to get depth: {error}")
            return None
    
    def get_kline_data(self, symbol: str, step: str = "15m", start_time=None, end_time=None):
        """
        取得 BitMart K 線資料
        :param symbol: 交易對 (例如 'ETHUSDT')
        :param step: K線時間粒度，可用 1m,3m,5m,15m,30m,1h,2h,4h,6h,12h,1d,3d,1w
        :param start_time: 起始時間（datetime 或 timestamp 或 None）
        :param end_time: 結束時間（datetime 或 timestamp 或 None）
        :return: pandas.DataFrame
        """

        # step 映射表（BitMart 需要整數分鐘）
        step_map = {
            "1m": 1, "3m": 3, "5m": 5, "15m": 15, "30m": 30,
            "1h": 60, "2h": 120, "4h": 240, "6h": 360, "12h": 720,
            "1d": 1440, "3d": 4320, "1w": 10080
        }

        if step not in step_map:
            raise ValueError(f"❌ 不支援的 step 格式: {step}")

        step_value = step_map[step]
        end_ts = self._time_to_timestamp(end_time)
        start_ts = self._time_to_timestamp(start_time) if start_time else end_ts - step_value * 60 * 500  # 最多 500 根

        url = f"{self.BASE_URL}/contract/public/kline"
        params = {
            "symbol": symbol,
            "step": step_value,
            "start_time": start_ts,
            "end_time": end_ts
        }

        try:
            r = requests.get(url, params=params)
            data = r.json()
            if data.get("code") != 1000:
                raise Exception(f"❌ API 回傳錯誤: {data}")

            df = pd.DataFrame(data["data"])
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
            df = df.rename(columns={
                "open_price": "open",
                "close_price": "close",
                "high_price": "high",
                "low_price": "low",
                "volume": "volume"
            })
            df = df[["timestamp", "open", "high", "low", "close", "volume"]]
            df = df.astype({
                "open": float, "high": float, "low": float,
                "close": float, "volume": float
            })
            return df.sort_values("timestamp").reset_index(drop=True)

        except Exception as e:
            print(f"🚨 取得 K 線資料失敗: {e}")
            return pd.DataFrame()
    
    def _get_price_precision(self, precision_str: str) -> int:
        """計算價格精度"""
        return len(precision_str.split('.')[1]) if '.' in precision_str else 0

    def _set_leverage(self, symbol: str, leverage: int):
        """設置逐倉杠桿"""
        try:
            self.futuresAPI.post_submit_leverage(
                contract_symbol=symbol,
                leverage=str(leverage),
                open_type="isolated"
            )
        except APIException as e:
            self.logger.warning(f"Failed to set leverage for {symbol}: {e}")
    
    def place_order(self, symbol: str, side: str, margin: float, leverage: int,
                    tp_price: float = None, sl_price: float = None):
        """
        下單函數 (兼容 SDK，固定逐倉，可選 TP/SL)
        :param symbol: 交易對
        :param side: 'long' 開多 或 'short' 開空
        :param margin: 使用保證金
        :param leverage: 杠桿倍數
        :param tp_price: 止盈價 (可選)
        :param sl_price: 止損價 (可選)
        :return: API 回應
        """

        # 1. 獲取當前價格
        current_price = self.get_current_price(symbol)
        if not current_price:
            self.logger.error(f"Cannot fetch current price for {symbol}")
            return None

        # 2. 獲取合約詳情
        try:
            details_data = self.futuresAPI.get_details(symbol)[0]['data']
            symbol_details = details_data['symbols'][0]
            contract_size = float(symbol_details['contract_size'])

            precision_str = symbol_details['price_precision']
            price_precision = len(precision_str.split('.')[1]) if '.' in precision_str else 0
        except (Exception) as e:
            self.logger.error(f"Could not get contract details: {e}")
            return None

        # 3. 計算下單數量
        size = int((margin * leverage) / (current_price * contract_size))
        if size <= 0:
            self.logger.error(f"Calculated size <= 0, cannot place order")
            return None

        # 4. 設置杠桿
        try:
            self.futuresAPI.post_submit_leverage(
                contract_symbol=symbol,
                leverage=str(leverage),
                open_type="isolated"
            )
        except Exception as e:
            self.logger.error(f"Failed to set leverage: {e}")

        # 5. 判斷下單方向
        side_map = {'long': 1, 'short': 4}  # 1=buy_open_long, 4=sell_open_short
        order_side = side_map.get(side.lower())
        if not order_side:
            self.logger.error(f"Invalid side: {side}")
            return None

        # 6. 四捨五入 TP/SL（如果有傳）
        payload_kwargs = {}
        if tp_price is not None:
            payload_kwargs['preset_take_profit_price'] = str(round(tp_price, price_precision))
        if sl_price is not None:
            payload_kwargs['preset_stop_loss_price'] = str(round(sl_price, price_precision))

        # 7. 發送下單請求
        try:
            response = self.futuresAPI.post_submit_order(
                contract_symbol=symbol,
                type="market",
                side=order_side,
                leverage=str(leverage),
                open_type="isolated",
                size=size,
                **payload_kwargs
            )
            return response
        except Exception as e:
            self.logger.error(f"Failed to submit order: {e}")
            return None
    
    def get_position(self, symbol: str):
        try:
            position_response = self.futuresAPI.get_position(symbol)
            positions = position_response[0]['data']
            for position in positions:
                if position['symbol'] == symbol:
                    return position
            self.logger.info(f"No open position found for {symbol}.")
            return None
        except (APIException, IndexError, KeyError) as e:
            self.logger.error(f"Failed to get position: {e}")
            return None

    
    def close_position(self, symbol: str):
        position = self.get_position(symbol)
        if not position:
            self.logger.info(f"No open position found for {symbol}.")
            return None

        position_type = position['position_type']
        current_amount = int(position['current_amount'])

        if current_amount == 0:
            self.logger.info(f"Position for {symbol} has size 0, nothing to close.")
            return None

        close_side = 0
        if position_type == 1: # Long position
            close_side = 3 # sell_close_long
        elif position_type == 2: # Short position (assuming it's 2, based on buy_close_short)
            close_side = 2 # buy_close_short
        else:
            self.logger.error(f"Unknown position type: {position_type}")
            return None

        try:
            leverage = position['leverage']
            open_type = position['margin_type'].lower()

            response = self.futuresAPI.post_submit_order(
                contract_symbol=symbol,
                type="market",
                side=close_side,
                size=current_amount,
                leverage=leverage,
                open_type=open_type
            )
            return response
        except (APIException, KeyError) as error:
            self.logger.error(f"Failed to close position: {error}")
            return None


# topone客戶端

In [12]:
import time
import hashlib
import requests
import logging
import json 

class TopOneClient:
    def __init__(self, api_key: str, secret_key: str, memo: str = None):
        self.api_key = api_key
        self.secret_key = secret_key
        self.memo = memo
        self.base_url = "https://openapi.top.one"
        self.logger = logging.getLogger(__name__)

    def _get_signed_headers(self, method, path):
        timestamp = str(int(time.time() * 1000))
        
        signature_payload = f"Method={method.upper()}&Path={path}&Timestamp={timestamp}&Secret={self.secret_key}"

        signature = hashlib.sha256(signature_payload.encode('utf-8')).hexdigest()

        return {
            "X-Time": timestamp,
            "X-Openapi-Key": self.api_key,
            "X-Openapi-Sign": signature,
            "Content-Type": "application/json"
        }

    def get_balance(self):
        path = "/api/v1/balance"
        method = "GET"
        
        headers = self._get_signed_headers(method, path)

        try:
            response = requests.get(self.base_url + path, headers=headers)
            response.raise_for_status()
            data = response.json()

            if data.get("status") and data.get("status").get("error") is None:
                trading_account = data.get("data", {}).get("trading", [])
                usdt_asset = next((asset for asset in trading_account if asset.get('code') == 'USDT'), None)
                if usdt_asset and 'available' in usdt_asset:
                    return float(usdt_asset['available'])
                else:
                    self.logger.info("USDT asset not found in trading account.")
                    return 0.0
            else:
                message = data.get("status", {}).get("messages", "Unknown error")
                self.logger.error(f"API error: {message}")
                return None

        except requests.exceptions.RequestException as e:
            self.logger.error(f"Request failed: {e}")
            return None
        except ValueError: # Catches JSON decoding errors
            self.logger.error("Failed to decode JSON response.")
            return None

    def place_order(self, symbol: str, side: str, margin: float, leverage: int, tp_price: float, sl_price: float):
        path = "/fapi/v1/create-order"
        method = "POST"

        headers = self._get_signed_headers(method, path)

        if side.lower() == 'long':
            api_side = "buy"
            api_position_side = "long"
        elif side.lower() == 'short':
            api_side = "sell"
            api_position_side = "short"
        else:
            self.logger.error(f"Invalid side: {side}. Must be 'long' or 'short'.")
            return None

        payload = {
            "pair": symbol,
            "side": api_side,
            "position_side": api_position_side,
            "leverage": leverage,
            "margin_mode": 1, #逐倉
            "margin": str(margin),
            "take_profit_price": str(tp_price),
            "stop_loss_price": str(sl_price),
        }

        try:
            response = requests.post(self.base_url + path, headers=headers, data=json.dumps(payload))
            response.raise_for_status()
            data = response.json()

            if data.get("status") and data.get("status").get("error") is None:
                return data
            else:
                message = data.get("status", {}).get("messages", "Unknown error")
                self.logger.error(f"API error: {message}")
                return None

        except requests.exceptions.RequestException as e:
            self.logger.error(f"Request failed: {e}")
            return None
        except ValueError: 
            self.logger.error("Failed to decode JSON response.")
            return None

    def get_open_positions(self, symbol: str = None):
        path = "/fapi/v1/position"
        method = "GET"
        
        headers = self._get_signed_headers(method, path)
        params = {"status": 1} # Filter for open positions

        if symbol:
            params["pair"] = symbol

        try:
            response = requests.get(self.base_url + path, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            if data.get("status") and data.get("status").get("error") is None:
                return data.get("data", {}).get("list", [])
            else:
                message = data.get("status", {}).get("messages", "Unknown error")
                self.logger.error(f"API error getting open positions: {message}")
                return None
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Request failed getting open positions: {e}")
            return None
        except ValueError:
            self.logger.error("Failed to decode JSON response getting open positions.")
            return None

    def get_position(self, symbol: str):
        """Get position for specific symbol (compatible with BitmartClient.get_position)"""
        try:
            positions = self.get_open_positions(symbol)
            if positions and len(positions) > 0:
                position = positions[0]
                #self.logger.info(f"Raw position data from get_open_positions for {symbol}: {position}")
                return {
                    'symbol': position.get('pair', symbol),
                    'size': position.get('quantity', '0'),
                    'side': position.get('side', None),  # TopOne uses 'side' field
                    'position_id': position.get('position_id'),
                    'entry_price': position.get('open_price', '0'),
                    'unrealized_pnl': position.get('unrealized_pnl', '0')
                }
            else:
                self.logger.info(f"No open position found for {symbol}.")
                return None
        except Exception as e:
            self.logger.error(f"Failed to get position for {symbol}: {e}")
            return None

    def close_position(self, symbol: str):
        open_positions = self.get_open_positions(symbol)
        if not open_positions:
            self.logger.info(f"No open positions found for {symbol}.")
            return None

        results = []
        for position in open_positions:
            position_id = position['position_id']
            quantity = position['quantity'] 

            path = "/fapi/v1/close"
            method = "POST"
            headers = self._get_signed_headers(method, path)
            payload = {
                "position_id": position_id,
                "quantity": quantity 
            }

            try:
                response = requests.post(self.base_url + path, headers=headers, data=json.dumps(payload))
                response.raise_for_status()
                data = response.json()

                if data.get("status") and data.get("status").get("error") is None:
                    self.logger.info(f"Position {position_id} closed successfully: {data}")
                    results.append({"position_id": position_id, "status": "success", "response": data})
                else:
                    message = data.get("status", {}).get("messages", "Unknown error")
                    self.logger.error(f"API error closing position {position_id}: {message}")
                    results.append({"position_id": position_id, "status": "failed", "message": message})
            except requests.exceptions.RequestException as e:
                self.logger.error(f"Request failed closing position {position_id}: {e}")
                results.append({"position_id": position_id, "status": "failed", "message": str(e)})
            except ValueError:
                self.logger.error(f"Failed to decode JSON response closing position {position_id}.")
                results.append({"position_id": position_id, "status": "failed", "message": "Invalid JSON response"})
        
        return results

# voger

In [13]:
import pandas as pd
import numpy as np

def voger_plus_signals(df, 
                       cciLen=20, atrLen=20, atrMult=1.0, 
                       smoothing=5, lookbackBars=5,
                       take_profit_trigger=0.001,   # 啟用追蹤止盈的浮盈%
                       trailing_stop_pct=0.0003):    # 回調多少%出場
  close = df['close']
  high = df['high']
  low = df['low']
  
  # === CCI ===
  TP = (high + low + close) / 3
  sma_tp = TP.rolling(cciLen, min_periods=cciLen).mean()
  mean_dev = TP.rolling(cciLen, min_periods=cciLen).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
  cci = (TP - sma_tp) / (0.015 * mean_dev)
  
  # === ATR (Wilder's RMA 版本) ===
  high_low = high - low
  high_close = np.abs(high - close.shift(1))
  low_close = np.abs(low - close.shift(1))
  tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
  atr = tr.ewm(alpha=1/atrLen, adjust=False).mean()  # RMA
  
  # === Trend Magic ===
  upTrend = close - atrMult * atr
  dnTrend = close + atrMult * atr
  rawTrendMagic = np.where(cci >= 0, upTrend, dnTrend)
  trendMagic = pd.Series(rawTrendMagic, index=df.index).rolling(smoothing, min_periods=smoothing).mean()
  
  trendUp = cci >= 0
  trendUpPrev = trendUp.shift(1).fillna(False).astype(bool)
  
  prevHigh = close.shift(1).rolling(lookbackBars).max()
  prevLow  = close.shift(1).rolling(lookbackBars).min()
  
  isAboveLookback = (lookbackBars == 0) | (close > prevHigh)
  isBelowLookback = (lookbackBars == 0) | (close < prevLow)
  
  bullCross = (~trendUpPrev) & trendUp & isAboveLookback
  bearCross = trendUpPrev & (~trendUp) & isBelowLookback
  
  signals = pd.Series(0, index=df.index)
  signals[bullCross] = 1
  signals[bearCross] = -1

  # === 追蹤止盈邏輯 ===
  position = 0       # 0: 無持倉, 1: 多單, -1: 空單
  entry_price = 0.0
  highest_profit_price = 0.0
  lowest_profit_price = 0.0

  for i in range(len(df)):
    sig = signals.iat[i]
    price = close.iat[i]
    
    # === 建倉 ===
    if sig == 1:
      position = 1
      entry_price = price
      highest_profit_price = price
      lowest_profit_price = price
    elif sig == -1:
      position = -1
      entry_price = price
      highest_profit_price = price
      lowest_profit_price = price

    # === 持倉期間 ===
    if position == 1:
      # 更新最高價
      highest_profit_price = max(highest_profit_price, price)
      profit_pct = (price - entry_price) / entry_price
      retrace_pct = (highest_profit_price - price) / highest_profit_price if highest_profit_price != 0 else 0
      
      if profit_pct >= take_profit_trigger and retrace_pct >= trailing_stop_pct:
        signals.iat[i] = 2   # 多單追蹤止盈出場
        position = 0

    elif position == -1:
      # 更新最低價
      lowest_profit_price = min(lowest_profit_price, price)
      profit_pct = (entry_price - price) / entry_price
      retrace_pct = (price - lowest_profit_price) / lowest_profit_price if lowest_profit_price != 0 else 0
      
      if profit_pct >= take_profit_trigger and retrace_pct >= trailing_stop_pct:
        signals.iat[i] = -2  # 空單追蹤止盈出場
        position = 0

  return signals


In [14]:
# 初始化 BitmartClient
bitmart_client = BitmartClient(BITMART_API_KEY, BITMART_SECRET_KEY, BITMART_MEMO)
# 查詢 K線
end_time = datetime.now()
start_time = datetime.now() - timedelta(days=4)
kline = bitmart_client.get_kline_data('ETHUSDT', "15m", start_time, end_time)
signal = voger_plus_signals(
    kline,
    take_profit_trigger=0.001,  # 啟動追蹤止盈的浮盈5%
    trailing_stop_pct=0.0003     # 回調2%出場
)
print(signal.value_counts())
signal 

 0    344
 1     15
-1     13
 2      7
-2      6
Name: count, dtype: int64


C:\Users\ricky\AppData\Local\Temp\ipykernel_18492\2329727233.py:33: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



0      0
1      0
2      0
3      0
4      0
      ..
380    0
381    0
382    0
383    0
384    1
Length: 385, dtype: int64

In [15]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# === 確保資料對齊 ===
kline = kline.reset_index(drop=True)
signal = pd.Series(signal).reset_index(drop=True)
if len(kline) != len(signal):
    print(f"⚠️ K線資料與信號長度不符 ({len(kline)} vs {len(signal)}), 以最短長度對齊")
    min_len = min(len(kline), len(signal))
    kline = kline.iloc[-min_len:].reset_index(drop=True)
    signal = signal.iloc[-min_len:].reset_index(drop=True)

kline['signal'] = signal

# === 建立圖表 ===
fig = go.Figure()

# --- K 線圖 ---
fig.add_trace(go.Candlestick(
    x=kline['timestamp'],
    open=kline['open'],
    high=kline['high'],
    low=kline['low'],
    close=kline['close'],
    name='K線',
    increasing_line_color='red',
    decreasing_line_color='green'
))

# --- 買入 (🚀) ---
buy_signals = kline[kline['signal'] == 1]
fig.add_trace(go.Scatter(
    x=buy_signals['timestamp'],
    y=buy_signals['low'] * 0.995,
    mode='text',
    text=['🚀'] * len(buy_signals),
    textposition="bottom center",
    name='開多'
))

# --- 賣出 (🪂) ---
sell_signals = kline[kline['signal'] == -1]
fig.add_trace(go.Scatter(
    x=sell_signals['timestamp'],
    y=sell_signals['high'] * 1.005,
    mode='text',
    text=['🪂'] * len(sell_signals),
    textposition="top center",
    name='開空'
))

# --- 多單追蹤止盈出場 (💰) ---
tp_long = kline[kline['signal'] == 2]
fig.add_trace(go.Scatter(
    x=tp_long['timestamp'],
    y=tp_long['high'] * 1.01,
    mode='text',
    text=['💰'] * len(tp_long),
    textposition="top center",
    name='多單追蹤止盈'
))

# --- 空單追蹤止盈出場 (💸) ---
tp_short = kline[kline['signal'] == -2]
fig.add_trace(go.Scatter(
    x=tp_short['timestamp'],
    y=tp_short['low'] * 0.99,
    mode='text',
    text=['💸'] * len(tp_short),
    textposition="bottom center",
    name='空單追蹤止盈'
))

# --- 外觀設定 ---
fig.update_layout(
    title='Voger+ 指標 K線圖（含買賣與追蹤止盈點）',
    xaxis_rangeslider_visible=True,
    height=600,
    margin=dict(l=50, r=50, t=80, b=50),
    template='plotly',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True)
)

fig.show()